In [1]:
#%autoreload
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
import numpy as np
import keras
import keras.backend as K
import tensorflow as tf
#import imp
#imp.reload(parse_data)

Using TensorFlow backend.


In [2]:
print keras.__version__

2.1.3


In [2]:
def mean_pred(y_true, y_pred):
    return keras.backend.l2_normalize(y_pred-y_true, axis=None)/keras.backend.l2_normalize(y_true, axis=None)
   # return np.linalg.norm(y_pred-y_true)/np.linalg.norm(y_true)

In [3]:
model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
input_x = Input((299,299,10,3))
x = Conv3D(3,(3,3,10), activation='relu', data_format="channels_last")(input_x)
x = ZeroPadding3D((1,1,0))(x)
x = Reshape((299,299,3))(x)
x = model_vgg16(x)

flatten = Flatten()
new_layer2 = Dense(1024, activation='relu', name='my_dense_2')
out2 = new_layer2(flatten(x))
x = Dense(512, activation='relu')(out2)
preds = Dense(3, activation = 'relu')(x)

model_start = Model(input_x, out2)
model_start.compile(loss='mean_squared_error', optimizer=SGD(lr=1e-4, momentum=0.9),
            metrics=[mean_pred])
model_start.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 299, 299, 10, 3)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 297, 297, 1, 3)    813       
_________________________________________________________________
zero_padding3d_1 (ZeroPaddin (None, 299, 299, 1, 3)    0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 299, 299, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 41472)             0         
_________________________________________________________________
my_dense_2 (Dense)           (None, 1024)              42468352  
Total para

In [33]:
object_path = "/home/zhouzixuan/proj/data/train/35/"
dev_path = "/home/zhouzixuan/proj/data/dev/35/"
filename ="vgg"
d_batch = np.load(object_path+str(19)+"_d.npy")
x_batch = np.load(object_path+str(19)+"_x.npy")
d_round = np.floor(d_batch/51.0)
sess = tf.InteractiveSession()
v = tf.transpose(tf.one_hot(d_round, depth=5, axis=2, on_value=1.0, off_value=0.0), perm=[0,1,3,2])
v = v.eval()
import time
combine = np.empty([32, 299, 299, 3, 0])
for i in range(5):
    i = 1
    v_tmp = v[:,:,:,i]
    v_tmp = np.transpose(np.broadcast_to(v_tmp,(3, 32, 299, 299)),(1,2,3,0))
    v_tmp = v_tmp == 1
    x_tmp = np.multiply(x_batch, v_tmp)
    x_cur = np.expand_dims(x_tmp, axis = 4)
    print combine.shape
    combine = np.concatenate((combine, x_cur), axis=4)
com = combine
combine = np.empty([32, 299, 299, 3, 0])

(32, 299, 299, 3, 0)
(32, 299, 299, 3, 1)
(32, 299, 299, 3, 2)
(32, 299, 299, 3, 3)
(32, 299, 299, 3, 4)
(32, 299, 299, 3, 0)
(32, 299, 299, 3, 1)
(32, 299, 299, 3, 2)
(32, 299, 299, 3, 3)
(32, 299, 299, 3, 4)
45574735.0
45574735.0


In [14]:
def train(model):

    fh = open(filename + 'report.txt','w')
    csv_logger = CSVLogger('logvgg.csv', append=True, separator=';')
    
    datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=False)
    
    train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=0.,
    width_shift_range=0.1,
    height_shift_range=0.1)
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow(x_train, y_train)
    validation_generator = test_datagen.flow(X_rgb, y_rgb)
    
    
    batch = 32
    epochs = 20
    batch_num = 60
    rescale=1. / 255
    for e in range(epochs):
        print('Epoch', e)
        batches = 0
        for b in range(batch_num):
            x_batch = np.load(object_path+str(b)+"_x.npy") * rescale
            d_batch = np.load(object_path+str(b)+"_d.npy")
            d_round = np.round(d_batch/25.5)
            
            y_batch = np.load(object_path+str(b)+"_y.npy")
            model.fit(x_batch, y_batch)
#         for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
#             x_batch = 
#             model.fit(x_batch, y_batch)
#             batches += 1
#             if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#                 break